In [0]:
# =============================================================================
# CONFIGURACIÓN DEL ENTORNO DATABRICKS SERVERLESS
# =============================================================================

from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys

print("🛠️ CONFIGURACIÓN DEL ENTORNO DATABRICKS SERVERLESS")
print("=" * 60)

# Información básica del sistema
config_info = [
    ("Versión de Spark", spark.version),
    ("Versión de Python", sys.version.split()[0]),
    ("Cluster Mode", "Serverless"),
    ("Disponible", "✅ Todo operativo")
]

print("\n📊 INFORMACIÓN DEL SISTEMA:")
print("-" * 40)
for key, value in config_info:
    print(f"  {key:<25} : {value}")

# Configuraciones de Spark
print("\n⚙️ CONFIGURACIONES SPARK DISPONIBLES:")
print("-" * 40)
configs_to_check = [
    "spark.sql.adaptive.enabled",
    "spark.databricks.clusterUsageTags.clusterName", 
]

for config in configs_to_check:
    try:
        value = spark.conf.get(config)
        print(f"  {config:<45} : {value}")
    except:
        print(f"  {config:<45} : No disponible")

# Verificación de almacenamiento
print("\n📁 ESTRUCTURA DE ALMACENAMIENTO:")
print("-" * 40)
try:
    files = dbutils.fs.ls("/FileStore")
    print(f"  FileStore contiene {len(files)} elementos")
    for file in files[:3]:  
        print(f"    📄 {file.name}")
    if len(files) > 3:
        print(f"    ... y {len(files) - 3} más")
except Exception as e:
    print(f"  ❌ FileStore no accesible: {e}")

🛠️ CONFIGURACIÓN DEL ENTORNO DATABRICKS SERVERLESS

📊 INFORMACIÓN DEL SISTEMA:
----------------------------------------
  Versión de Spark          : 4.0.0
  Versión de Python         : 3.12.3
  Cluster Mode              : Serverless
  Disponible                : ✅ Todo operativo

⚙️ CONFIGURACIONES SPARK DISPONIBLES:
----------------------------------------
  spark.sql.adaptive.enabled                    : No disponible
  spark.databricks.clusterUsageTags.clusterName : No disponible

📁 ESTRUCTURA DE ALMACENAMIENTO:
----------------------------------------
  ❌ FileStore no accesible: Public DBFS root is disabled. Access is denied on path: /FileStore

JVM stacktrace:
java.lang.UnsupportedOperationException
	at com.databricks.backend.daemon.data.client.DisabledDatabricksFileSystem.rejectOperation(DisabledDatabricksFileSystem.scala:31)
	at com.databricks.backend.daemon.data.client.DisabledDatabricksFileSystem.listStatus(DisabledDatabricksFileSystem.scala:96)
	at com.databricks.backend.dae

In [0]:
# =============================================================================
# CONEXIÓN Y ANÁLISIS INICIAL DEL DATASET
# =============================================================================

print("\n📊 CONEXIÓN AL DATASET TITANIC")
print("=" * 60)

# Cargar datos
df = spark.table("default.Train")

# Estadísticas básicas con formato
total_records = df.count()
total_columns = len(df.columns)

print(f"\n✅ CONEXIÓN EXITOSA A: default.Train")
print("-" * 40)
print(f"  📈 Total de registros : {total_records:,}")
print(f"  🗂️  Total de columnas  : {total_columns}")
print(f"  🔍 Muestra de datos   :")

# Mostrar datos con mejor formato
print("\n👥 PRIMEROS 5 PASAJEROS:")
print("-" * 60)
display(df.limit(5))


📊 CONEXIÓN AL DATASET TITANIC

✅ CONEXIÓN EXITOSA A: default.Train
----------------------------------------
  📈 Total de registros : 891
  🗂️  Total de columnas  : 12
  🔍 Muestra de datos   :

👥 PRIMEROS 5 PASAJEROS:
------------------------------------------------------------


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,null,S


In [0]:
print("\n📋 ANÁLISIS DEL ESQUEMA DE DATOS")
print("=" * 60)

print("\n🔍 ESQUEMA DETALLADO:")
print("-" * 40)
df.printSchema()

print("\n📝 DESCRIPCIÓN DE LA TABLA:")
print("-" * 40)
# Crear una vista mejorada del describe
desc_df = spark.sql("DESCRIBE default.Train")
display(desc_df)


📋 ANÁLISIS DEL ESQUEMA DE DATOS

🔍 ESQUEMA DETALLADO:
----------------------------------------
root
 |-- PassengerId: long (nullable = true)
 |-- Survived: long (nullable = true)
 |-- Pclass: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: long (nullable = true)
 |-- Parch: long (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)


📝 DESCRIPCIÓN DE LA TABLA:
----------------------------------------


col_name,data_type,comment
PassengerId,bigint,null
Survived,bigint,null
Pclass,bigint,null
Name,string,null
Sex,string,null
Age,double,null
SibSp,bigint,null
Parch,bigint,null
Ticket,string,null
Fare,double,null


In [0]:
# =============================================================================
# VALIDACIONES ESTADÍSTICAS Y DE CALIDAD
# =============================================================================

print("\n🔍 VALIDACIONES ESTADÍSTICAS")
print("=" * 60)

# Estadísticas descriptivas con mejor formato
print("\n📊 ESTADÍSTICAS DESCRIPTIVAS:")
print("-" * 40)
stats_df = df.describe()
display(stats_df)

# Distribución por clase con formato
print("\n🎫 DISTRIBUCIÓN POR CLASE:")
print("-" * 40)
class_distribution = df.groupBy("Pclass").agg(
    count("*").alias("total_pasajeros"),
    expr("ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM default.Train), 1)").alias("porcentaje")
).orderBy("Pclass")

display(class_distribution)

# Análisis de supervivencia básico
print("\n🆘 ANÁLISIS DE SUPERVIVENCIA GENERAL:")
print("-" * 40)
survival_stats = df.agg(
    count("*").alias("total_pasajeros"),
    expr("SUM(Survived)").alias("sobrevivientes"),
    expr("ROUND(AVG(Survived) * 100, 2)").alias("tasa_supervivencia_%")
)
display(survival_stats)


🔍 VALIDACIONES ESTADÍSTICAS

📊 ESTADÍSTICAS DESCRIPTIVAS:
----------------------------------------


summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,null,null,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,null,null
stddev,257.3538420152301,0.4865924542648585,0.8360712409770513,null,null,14.526497332334044,1.1027434322934275,0.8060572211299559,471609.26868834946,49.693428597180905,null,null
min,1,0,1,"Abbing, Mr. Anthony",female,0.42,0,0,110152,0.0,A10,C
max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S



🎫 DISTRIBUCIÓN POR CLASE:
----------------------------------------


Pclass,total_pasajeros,porcentaje
1,216,24.2
2,184,20.7
3,491,55.1



🆘 ANÁLISIS DE SUPERVIVENCIA GENERAL:
----------------------------------------


total_pasajeros,sobrevivientes,tasa_supervivencia_%
891,342,38.38


In [0]:
print("\n👥 ANÁLISIS POR GÉNERO Y CLASE")
print("=" * 60)

# Análisis detallado con PySpark
analysis_pyspark = df.groupBy("Pclass", "Sex").agg(
    count("*").alias("total_pasajeros"),
    sum("Survived").alias("sobrevivientes"),
    expr("ROUND(AVG(Survived) * 100, 1)").alias("tasa_supervivencia_%"),
    expr("ROUND(AVG(Fare), 2)").alias("tarifa_promedio")
).orderBy("Pclass", "Sex")

print("\n🔄 RESULTADO CON PYSPARK:")
print("-" * 40)
display(analysis_pyspark)

# Crear vista temporal para SQL
df.createOrReplaceTempView("titanic_view")


👥 ANÁLISIS POR GÉNERO Y CLASE

🔄 RESULTADO CON PYSPARK:
----------------------------------------


Pclass,Sex,total_pasajeros,sobrevivientes,tasa_supervivencia_%,tarifa_promedio
1,female,94,91,96.8,106.13
1,male,122,45,36.9,67.23
2,female,76,70,92.1,21.97
2,male,108,17,15.7,19.74
3,female,144,72,50.0,16.12
3,male,347,47,13.5,12.66


In [0]:
%sql
-- Consultas SQL funcionan perfectamente en Serverless
-- 1. Metadatos de la tabla
DESCRIBE default.Train;

-- 2. Estadísticas básicas
SELECT 
    COUNT(*) as total_pasajeros,
    AVG(Survived) as tasa_supervivencia,
    AVG(Age) as edad_promedio,
    AVG(Fare) as tarifa_promedio
FROM default.Train;

-- 3. Supervivencia por clase
SELECT 
    Pclass,
    COUNT(*) as total,
    AVG(Survived) as tasa_supervivencia
FROM default.Train 
GROUP BY Pclass 
ORDER BY Pclass;

Pclass,total,tasa_supervivencia
1,216,0.6296296296296297
2,184,0.47282608695652173
3,491,0.24236252545824846


In [0]:

print("\n🌐 ANÁLISIS MULTIDIMENSIONAL: CLASE + GÉNERO + EMBARQUE")
print("=" * 60)

# Análisis complejo con filtros y ordenamiento
multidimensional_analysis = (df
    .groupBy("Pclass", "Sex", "Embarked")
    .agg(
        count("*").alias("total_pasajeros"),
        sum("Survived").alias("sobrevivientes"),
        expr("ROUND(AVG(Survived) * 100, 1)").alias("tasa_supervivencia_%"),
        expr("ROUND(AVG(Age), 1)").alias("edad_promedio"),
        expr("ROUND(AVG(Fare), 2)").alias("tarifa_promedio")
    )
    .filter(col("total_pasajeros") > 10)  # Filtrar grupos significativos
    .orderBy("Pclass", "Sex", "Embarked")
)

print("\n📊 RESULTADOS MULTIDIMENSIONALES:")
print("-" * 40)
display(multidimensional_analysis)

# Resumen ejecutivo
print("\n✅ RESUMEN EJECUTIVO:")
print("-" * 40)
print(f"• Se analizaron {total_records:,} registros del Titanic")
print(f"• {multidimensional_analysis.count()} combinaciones significativas encontradas")
print(f"• Análisis completado exitosamente")


🌐 ANÁLISIS MULTIDIMENSIONAL: CLASE + GÉNERO + EMBARQUE

📊 RESULTADOS MULTIDIMENSIONALES:
----------------------------------------


Pclass,Sex,Embarked,total_pasajeros,sobrevivientes,tasa_supervivencia_%,edad_promedio,tarifa_promedio
1,female,C,43,42,97.7,36.1,115.64
1,female,S,48,46,95.8,32.7,99.03
1,male,C,42,17,40.5,40.1,93.54
1,male,S,79,28,35.4,41.9,52.95
2,female,S,67,61,91.0,29.7,21.91
2,male,S,97,15,15.5,30.9,19.23
3,female,C,23,15,65.2,14.1,14.69
3,female,Q,33,24,72.7,22.9,10.31
3,female,S,88,33,37.5,23.2,18.67
3,male,C,43,10,23.3,25.0,9.35



✅ RESUMEN EJECUTIVO:
----------------------------------------
• Se analizaron 891 registros del Titanic
• 12 combinaciones significativas encontradas
• Análisis completado exitosamente


## Comparación: SQL vs Spark

### Ventajas de SQL
- **Sintaxis familiar**: Muchos analistas de datos conocen SQL.
- **Declarativo**: Especificas qué quieres, no cómo obtenerlo.
- **Optimización**: El motor de SQL puede optimizar la consulta automáticamente.
- **Integración**: Fácil integración con herramientas de BI.

### Desventajas de SQL
- **Limitaciones en transformaciones complejas**: Algunas transformaciones son más difíciles de expresar en SQL.
- **Menos flexibilidad**: Para pipelines complejos, puede ser menos flexible que Spark.

### Ventajas de Spark (PySpark)
- **Escalabilidad**: Diseñado para big data y procesamiento distribuido.
- **APIs ricas**: DataFrame y Dataset APIs para transformaciones complejas.
- **UDFs**: Permite definir funciones personalizadas en Python, Scala, etc.
- **Integración con MLlib**: Para machine learning.

### Desventajas de Spark
- **Curva de aprendizaje**: Requiere aprender nuevas APIs y conceptos.
- **Configuración**: Puede requerir ajustes de rendimiento.
- **Overhead**: Para consultas simples, puede ser más lento que SQL.

### Conclusión
En entornos como Databricks, podemos aprovechar lo mejor de ambos: usar SQL para consultas ad-hoc y PySpark para pipelines de datos más complejos. La elección depende del caso de uso y de la familiaridad del equipo con cada tecnología.

## Diseño del Esquema - Dataset Titanic

### Diccionario de Datos
| Columna | Tipo | Descripción | Nulable |
|---------|------|-------------|---------|
| PassengerId | BIGINT | ID único del pasajero | NO |
| Survived | BIGINT | Supervivencia (0=No, 1=Sí) | SÍ |
| Pclass | BIGINT | Clase del ticket (1,2,3) | SÍ |
| Name | STRING | Nombre completo | SÍ |
| Sex | STRING | Género | SÍ |
| Age | DOUBLE | Edad | SÍ |
| SibSp | BIGINT | Hermanos/Esposos a bordo | SÍ |
| Parch | BIGINT | Padres/Hijos a bordo | SÍ |
| Ticket | STRING | Número de ticket | SÍ |
| Fare | DOUBLE | Tarifa pagada | SÍ |
| Cabin | STRING | Cabina | SÍ |
| Embarked | STRING | Puerto de embarque | SÍ |

**Llave primaria:** PassengerId

In [0]:
print("=== ANÁLISIS POR GRUPOS DE EDAD ===")

from pyspark.sql.functions import when, col

# Crear grupos de edad
df_age_groups = df.withColumn(
    "age_group",
    when(col("Age").isNull(), "Desconocido")
    .when(col("Age") < 18, "Niño")
    .when(col("Age") < 30, "Joven")
    .when(col("Age") < 50, "Adulto")
    .otherwise("Mayor")
)

# Análisis de supervivencia por grupo de edad
age_analysis = df_age_groups.groupBy("age_group") \
    .agg(
        count("*").alias("total"),
        avg("Survived").alias("tasa_supervivencia"),
        avg("Fare").alias("tarifa_promedio")
    ) \
    .orderBy("tasa_supervivencia", ascending=False)

print("Supervivencia por Grupo de Edad:")
age_analysis.show()

=== ANÁLISIS POR GRUPOS DE EDAD ===
Supervivencia por Grupo de Edad:
+-----------+-----+-------------------+------------------+
|  age_group|total| tasa_supervivencia|   tarifa_promedio|
+-----------+-----+-------------------+------------------+
|       Niño|  113| 0.5398230088495575| 31.22079823008851|
|     Adulto|  256|         0.41796875|   39.551611328125|
|      Mayor|   74|0.36486486486486486| 46.36441486486488|
|      Joven|  271| 0.3505535055350554|28.368094464944658|
|Desconocido|  177| 0.2937853107344633|22.158566666666673|
+-----------+-----+-------------------+------------------+



In [0]:
%sql
-- Análisis por grupos de edad con SQL
SELECT 
    CASE 
        WHEN Age IS NULL THEN 'Desconocido'
        WHEN Age < 18 THEN 'Niño'
        WHEN Age < 30 THEN 'Joven' 
        WHEN Age < 50 THEN 'Adulto'
        ELSE 'Mayor'
    END as age_group,
    COUNT(*) as total,
    AVG(Survived) as tasa_supervivencia,
    AVG(Fare) as tarifa_promedio
FROM default.Train 
GROUP BY 
    CASE 
        WHEN Age IS NULL THEN 'Desconocido'
        WHEN Age < 18 THEN 'Niño'
        WHEN Age < 30 THEN 'Joven'
        WHEN Age < 50 THEN 'Adulto'
        ELSE 'Mayor'
    END
ORDER BY tasa_supervivencia DESC;

age_group,total,tasa_supervivencia,tarifa_promedio
Niño,113,0.5398230088495575,31.22079823008851
Adulto,256,0.41796875,39.551611328125
Mayor,74,0.36486486486486486,46.36441486486488
Joven,271,0.3505535055350554,28.368094464944658
Desconocido,177,0.2937853107344633,22.158566666666673


In [0]:
print("=== ANÁLISIS COMPLEJO: CLASE + GÉNERO + EMBARQUE ===")

complex_analysis = (df
    .groupBy("Pclass", "Sex", "Embarked")
    .agg(
        count("*").alias("total_pasajeros"),
        avg("Survived").alias("tasa_supervivencia"),
        avg("Age").alias("edad_promedio"),
        avg("Fare").alias("tarifa_promedio")
    )
    .filter(col("total_pasajeros") > 10)   # Filtrar grupos pequeños
    .orderBy("Pclass", "Sex", "Embarked")
)

print("Análisis Multidimensional:")
complex_analysis.show()

=== ANÁLISIS COMPLEJO: CLASE + GÉNERO + EMBARQUE ===
Análisis Multidimensional:
+------+------+--------+---------------+-------------------+------------------+------------------+
|Pclass|   Sex|Embarked|total_pasajeros| tasa_supervivencia|     edad_promedio|   tarifa_promedio|
+------+------+--------+---------------+-------------------+------------------+------------------+
|     1|female|       C|             43| 0.9767441860465116| 36.05263157894737| 115.6403093023256|
|     1|female|       S|             48| 0.9583333333333334| 32.70454545454545| 99.02691041666664|
|     1|  male|       C|             42|0.40476190476190477|40.111111111111114| 93.53670714285715|
|     1|  male|       S|             79|0.35443037974683544|        41.8971875| 52.94994683544305|
|     2|female|       S|             67| 0.9104477611940298| 29.71969696969697| 21.91268656716418|
|     2|  male|       S|             97|0.15463917525773196|30.875888888888888|19.232474226804122|
|     3|female|       C|     